# Задание 7

## 1. PropertyCreator (0.2 балла)

Напишите мета класс для создания свойств (property) класса из функций начинающихся с "set\_", "get\_" или "del_". Пример использования:
<code>
class TestPropertyCreator(metaclass=PropertyCreator):
    def \__init\__(self, lo):
        self.__x = None
        self.lo = lo

    def get_x(self):
        return self.__x

    def set_x(self, value):
        if value < self.lo:
            raise ValueError("Value must in condition: {} <= value".format(self.lo))
        self.__x = value
    
    def del_x(self):
        self.__x = "No more"

    pass


obj = TestPropertyCreator(5)
obj.x = 4
print(obj.x)
del (obj.x)
</code>

* Мета класс должен поддерживать наследование, в смысле создавать свойства у потомков.
* Должен поддерживать частичное описание свойств, т. е. например, описание одного метода get_val (без set_val и del_val).
* Поддерживать множественное использование одного свойства с одним именем в разных классах:

<code>
class A(metaclass=PropertyCreator):
    def get_x(self):
        return "x in class A"

class B(metaclass=PropertyCreator):
    def get_x(self):
        return "x in class B"

class C(metaclass=PropertyCreator):
    def set_x(self, value):
        self.value = "x in class C"
    def get_x(self):
        return self.value
</code>
* Должен уметь обрабатывать имен с несколькими подчеркиваниями "get_raw_text".

#### Решение

In [43]:
class PropertyCreator(type):
    def get_name(name):
        '''get_abc_123 -> abc_123'''
        return '_'.join(name.split('_')[1:])
    
    def raise_(e):
        raise e
        
    default_getter = lambda x: PropertyCreator.raise_(AttributeError('no getter'))
    default_setter = lambda x, y: PropertyCreator.raise_(AttributeError('no setter'))
    default_deleter = lambda x: PropertyCreator.raise_(AttributeError('no deleter'))

    def __new__(cls, name, bases, dct):    
        properties = set()
        new_dct = {}
        for name, value in dct.items():
            # добавляем в список свойств 
            if name.split('_')[0] in ['get', 'set', 'del']:
                properties.add(PropertyCreator.get_name(name))
            else:
                new_dct[name] = value
                
        for prop in properties:
            new_dct[prop] = property(
                dct.get(f'get_{prop}', PropertyCreator.default_getter),
                dct.get(f'set_{prop}', PropertyCreator.default_setter),
                dct.get(f'del_{prop}', PropertyCreator.default_deleter)
            )
        
        return type.__new__(cls, name, bases, new_dct)

#### Протестируйте свое решение

In [63]:
def test_simple():
    class TestPropertyCreator(metaclass=PropertyCreator):
        def __init__(self, lo):
            self.__x = None
            self.lo = lo
        def get_x(self):
            return self.__x

        def set_x(self, value):
            if value < self.lo:
                raise ValueError("Value must in condition: {} <= value".format(self.lo))
            self.__x = value

        def del_x(self):
            self.__x = "No more"
            
    t = TestPropertyCreator(0)
    t.x = 500
    print(t.x)
    del(t.x)
    print(t.x)
    try:
        t.x = -1
    except ValueError as e:
        print(e)
    
    
def test_with_inheritance():
    class TestPropertyCreator(metaclass=PropertyCreator):
        pass

    class TestPropertyCreatorInheritance(TestPropertyCreator):
        def __init__(self):
            self._secret_list = []

        def get_x(self):
            self._secret_list.append("get")
            return 0

        def set_x(self, value):
            self._secret_list.append("set")
            
    t = TestPropertyCreatorInheritance()
    print(t.x)
    t.x = 322
    print(t.x)
    print(t._secret_list)

    
def test_partially_defined():
    class TestPropertyCreator(metaclass=PropertyCreator):
        def __init__(self):
            self._secret_list = []

        def get_x(self):
            self._secret_list.append("get")
            return 0

        def set_y(self, value):
            self._secret_list.append("set")
            self._y = value
    
    t = TestPropertyCreator()
    print(t.x)
    try:
        t.x = 1
    except AttributeError as e:
        print(e)
    t.y = 2
    try:
        print(t.y)
    except AttributeError as e:
        print(e)
    
    try:
        del t.x
    except AttributeError as e:
        print(e)
    print(t._secret_list)
    

    
def test_sanity():
    class TestPropertyCreator(metaclass=PropertyCreator):
        _text = 0
        def get_raw_text(self):
             return self._text

        def get_text(self):
             return self._text % 2

        def set_text(self, value):
            try:
                self._text = int(value)
            except ValueError:
                raise TypeError("unproper value for text: {}".format(value))

    t = TestPropertyCreator()
    print(t.raw_text)
    t.text = 9
    print(t.raw_text)
    print(t.text)
    try:
        t.text = 'abc'
    except TypeError as e:
        print(e)
    print(t.text)


def test_multiple_usages():
    class TestPropertyCreatorA(metaclass=PropertyCreator):
        def get_x(self):
            return 0
    class TestPropertyCreatorB(metaclass=PropertyCreator):
        def get_x(self):
            return 1
    class TestPropertyCreatorC(metaclass=PropertyCreator):
        def set_x(self, value):
            self.value = value + 1
        def get_x(self):
            return self.value

    ta = TestPropertyCreatorA()
    tb = TestPropertyCreatorB()
    tc = TestPropertyCreatorC()
    tcc = TestPropertyCreatorC()
    print(ta.x, tb.x)
    tc.x = 10
    tcc.x = 15
    print(tc.x, tcc.x)
    print(ta.x, tb.x)


In [64]:
test_simple()

500
No more
Value must in condition: 0 <= value


In [65]:
test_with_inheritance()

0
0
['get', 'set', 'get']


In [66]:
test_partially_defined()

0
no setter
no getter
no deleter
['get', 'set']


In [67]:
test_sanity()

0
9
1
unproper value for text: abc
1


In [68]:
test_multiple_usages()

0 1
11 16
0 1


## 2. InstanceCountExeptioner (0.2 балла)
Напишите метакласс InstanceCountExeptioner, который будет следить за количеством экземпляров класса, использующих его. Количество задается через поле класса \_\_max_instane\_count\_\_. Т. е. число экземпляров каждого класса регулируется отдельно. Если в классе не указано поле \_\_max_instane\_count\_\_, то используйте заранее заданное число в метаклассе (любое). Пример:

<code>
class TestInstanceCountExeptionerA(metaclass=InstanceCountExeptioner):
    \_\_max_instane\_count\_\_ = 2
    def \__init\__(self, a):
        self.a = a


class TestInstanceCountExeptionerB(metaclass=InstanceCountExeptioner):
    \_\_max_instane\_count\_\_ = 1
    def \__init\__(self, a):
        self.a = a

a_one = TestInstanceCountExeptionerA('one')
a_two = TestInstanceCountExeptionerA('two')
b_one = TestInstanceCountExeptionerB('one')
\# пока всё шло хорошо

\# а вот
a_three = TestInstanceCountExeptionerA('three')
\# выкенет исключение InstanceCountExeption (ваше собственное исключение)
</code>

#### Решение

In [206]:
# Опишите исключение InstanceCountException
class InstanceCountException(Exception):
    pass

In [452]:
# Опишите мета класс InstanceCountExceptioner
class InstanceCountExceptioner(type):
    instance_count = {}
    __max_instance_count__ = 2
    
    def __call__(cls, *args, **kwargs):
        if '__max_instance_count__' in cls.__dict__:
            cls_max_count = cls.__max_instance_count__
        else:
            cls_max_count = InstanceCountExceptioner.__max_instance_count__
            
        name = cls.__name__
            
        if name not in InstanceCountExceptioner.instance_count:
            InstanceCountExceptioner.instance_count[name] = 0

        InstanceCountExceptioner.instance_count[name] += 1
        if InstanceCountExceptioner.instance_count[name] > cls_max_count:
            raise InstanceCountException(
                f'Слишком много объектов класса {name}, максимум: {cls_max_count}'
            )
        
        return type.__call__(cls, *args, **kwargs)

#### Протестируйте свое решение

In [219]:
class TestInstanceCountExceptionerA(metaclass=InstanceCountExceptioner):
    __max_instance_count__ = 2

    def __init__(self):
        self.a = 1

    def get(self):
        return self.a


class TestInstanceCountExceptionerB(metaclass=InstanceCountExceptioner):
    __max_instance_count__ = 3

    def __init__(self):
        self.b = 2

    def get(self):
        return self.b

    
def test_simple():
    ta = TestInstanceCountExceptionerA()
    tb = TestInstanceCountExceptionerB()
    print(ta.get())
    print(tb.get())
    

def test_create():
    ta = TestInstanceCountExceptionerA()
    tb = TestInstanceCountExceptionerB()
    taa = TestInstanceCountExceptionerA()
    tbb = TestInstanceCountExceptionerB()
    tbbb = TestInstanceCountExceptionerB()
    print(taa.get())
    print(tbbb.get())
    
def test_fail_create_a():
    ta = TestInstanceCountExceptionerA()
    taa = TestInstanceCountExceptionerA()
    print(ta.get(), taa.get())
    try:
        taaa = TestInstanceCountExceptionerA()
    except InstanceCountException as e:
        print(e)
    

def test_fail_create_b():
    tb = TestInstanceCountExceptionerB()
    tbb = TestInstanceCountExceptionerB()
    tbbb = TestInstanceCountExceptionerB()

    print(tb.get(), tbb.get(), tbbb.get())
    try:
        tbbbb = TestInstanceCountExceptionerB()
    except InstanceCountException as e:
        print(e)


После каждого теста класс `InstanceCountExceptioner` определялся заново, иначе он бы считал объекты из разных тестов.

In [209]:
test_simple()

1
2


In [214]:
test_create()

1
2


In [217]:
test_fail_create_a()

1 1
Слишком много объектов класса TestInstanceCountExceptionerA, максимум: 2


In [220]:
test_fail_create_b()

2 2 2
Слишком много объектов класса TestInstanceCountExceptionerB, максимум: 3


In [453]:
class TestInstanceCountExceptionerC(metaclass=InstanceCountExceptioner):
    # не задаём в классе __max_instance_count__, оно берётся из InstanceCountExceptioner
    def __init__(self):
        self.c = 2

    def get(self):
        return self.c
    
tc = TestInstanceCountExceptionerC()
tcc = TestInstanceCountExceptionerC()

print(tc.get(), tcc.get())
try:
    tccc = TestInstanceCountExceptionerC()
except InstanceCountException as e:
    print(e)

2 2
Слишком много объектов класса TestInstanceCountExceptionerC, максимум: 2


## 3. JSONClassCreator (0.6 баллов)
Напишите метакласс, который будет по json представлению строить новый класс и обратно. Класс должен уметь следующее:
* Поддерживать сохранение и получение магических функций класса.
* Поддерживать сохранение и получение обычных функций.
* Поддерживать сохранение полей со стандартными типами.
* Уберите из сохранения следующие поля и методы: ['\_\_dict\_\_', '\_\_weakref\_\_', '\_\_module\_\_', '\_\_init\_\_']
* У создаваемого класса должна быть функция to_json_str

Формат json строки должен быть следующий:

<code>
{
    "name": название класса,
    "bases": базовые классы,
    "methods": методы класса,
    "attrs": поля класса
}
</code>

Рекомендации:
* Для получения кода функций используйте модуль <a href="http://python-lab.ru/documentation/27/stdlib/inspect.html">inspect</a>.
* Для того, чтобы запустить код функций, можно использовать exec.
* Можно не исправлять ошибку типа OSError: could not get source code - возникает для функций, полученных с помощью exec.

#### Пример использования

In [3]:
import json


class ParentTest1(object):
    pass

class ParentTest2(object):
    pass

class Test(ParentTest1, ParentTest2):
    """Тестовый класс"""

    val = [1, 2, 3]

    def f(self, x):
        print(x)
    
    def __repr__(self):
        return "Test(val={})".format(self.val)

    def __str__(self):
        return "Test(val={})".format(self.val)

    pass

In [5]:
print(*json.loads(JSONClassCreator.to_json_str(Test)).items(), sep="\n")

('name', 'Test')
('bases', ['ParentTest1', 'ParentTest2'])
('methods', {'f': 'def f(self, x):\n        print(x)', '__repr__': 'def __repr__(self):\n        return "Test(val={})".format(self.val)', '__str__': 'def __str__(self):\n        return "Test(val={})".format(self.val)'})
('attrs', {'__doc__': 'Тестовый класс', 'val': [1, 2, 3]})


In [6]:
tmp = JSONClassCreator(JSONClassCreator.to_json_str(Test))

In [7]:
tmp_obj = tmp()
tmp_obj, tmp_obj.f("hi"), tmp.val, tmp.__doc__

hi


(Test(val=[1, 2, 3]), None, [1, 2, 3], 'Тестовый класс')

In [10]:
tmp.__dict__

mappingproxy({'__doc__': 'Тестовый класс',
              '__module__': '__main__',
              '__repr__': <function __main__.__repr__>,
              '__str__': <function __main__.__str__>,
              'f': <function __main__.f>,
              'to_json_str': <function __main__.JSONClassCreator.to_json_str>,
              'val': [1, 2, 3]})

#### Решение

In [4]:
import inspect
import json

class JSONClassCreator(type):
    def __new__(mcls, json_str):
        cls_dict = json.loads(json_str)
        name = cls_dict['name']
        bases = tuple([globals()[base] for base in cls_dict['bases']])
        
        # я нашёл только такой способ получить функции по их коду
        for _, code in cls_dict['methods'].items():
#             exec(code, globals(), locals())
            exec(code, globals())

        
        methods = {name: globals()[name]
                   for name, _ in cls_dict['methods'].items()}
                
        attrs = cls_dict['attrs']
        return type.__new__(mcls, name, bases, {**methods, **attrs})
        

    def to_json_str(cls):
        exclude = ['__dict__', '__weakref__', '__module__', '__init__']
        # всё что callable
        methods = {name: inspect.getsource(value).strip()
                   for name, value in cls.__dict__.items() 
                   if callable(value) and name not in exclude}
        
        # всё что не callable
        attrs = {name: value for name, value in cls.__dict__.items() 
                   if not callable(value) and name not in exclude}
        
        if hasattr(cls, '__bases__'):
            bases = [base.__name__ for base in cls.__bases__]
        else:
            bases = []
            
        return json.dumps({
            "name": cls.__name__,
            "bases": bases,
            "methods": methods,
            "attrs": attrs
        })



#### Проверьте свое решение на примере

In [5]:
print(*json.loads(JSONClassCreator.to_json_str(Test)).items(), sep="\n")

('name', 'Test')
('bases', ['ParentTest1', 'ParentTest2'])
('methods', {'f': 'def f(self, x):\n        print(x)', '__repr__': 'def __repr__(self):\n        return "Test(val={})".format(self.val)', '__str__': 'def __str__(self):\n        return "Test(val={})".format(self.val)'})
('attrs', {'__doc__': 'Тестовый класс', 'val': [1, 2, 3]})


In [6]:
tmp = JSONClassCreator(JSONClassCreator.to_json_str(Test))

In [7]:
tmp_obj = tmp()
tmp_obj, tmp_obj.f("hi"), tmp.val, tmp.__doc__

hi


(Test(val=[1, 2, 3]), None, [1, 2, 3], 'Тестовый класс')

In [8]:
tmp.__dict__

mappingproxy({'f': <function __main__.f(self, x)>,
              '__repr__': <function __main__.__repr__(self)>,
              '__str__': <function __main__.__str__(self)>,
              '__doc__': 'Тестовый класс',
              'val': [1, 2, 3],
              '__module__': '__main__'})